In [41]:
import numpy as np
import cv2

In [42]:
import cv2
image_file = "temp/page_01.jpg"
img =  cv2.imread(image_file)

In [43]:
cv2.imshow("original image", img)
cv2.waitKey(0)

-1

Inverted Image

In [44]:
inverted_image = cv2.bitwise_not(img)
cv2.imwrite("temp/inverted.jpg", inverted_image)

True

In [45]:
img = cv2.imread("temp/inverted.jpg")
cv2.imshow("inverted", img)
cv2.waitKey(0)

-1

# 02: Rescaling

# 03: Binarization

In [46]:
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [47]:
img = cv2.imread("temp/page_01.jpg")
gray_image = grayscale(img)
cv2.imwrite("temp/gray2.jpg", gray_image)

True

In [48]:
img = cv2.imread("temp/gray2.jpg")
cv2.imshow("gray", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [49]:
thresh, im_bw = cv2.threshold(gray_image, 210, 230, cv2.THRESH_BINARY)
cv2.imwrite("temp/bw_image.jpg", im_bw)

True

In [55]:
img = cv2.imread("temp/bw_image.jpg")
cv2.imshow("gray", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Noise Removal

In [52]:
def noise_removal(image):
    kernel = np.ones((1,1), np.uint8)
    image = cv2.dilate(image, kernel, iterations= 1)
    kernel = np.ones((1,1), np.uint8)
    image = cv2.erode(image, kernel, iterations= 1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return(image)

In [53]:
no_noise = noise_removal(im_bw)
cv2.imwrite("temp/no_noise.jpg", no_noise)

True

In [57]:
img = cv2.imread("temp/no_noise.jpg")
cv2.imshow("gray", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Erosão

In [58]:
def thin_font(image):
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.erode(image, kernel, iterations = 1)
    image = cv2.bitwise_not(image)
    return(image)

In [59]:
eroded_image = thin_font(no_noise)
cv2.imwrite("temp/erodede_image.jpg", eroded_image)

True

In [60]:
img = cv2.imread("temp/erodede_image.jpg")
cv2.imshow("gray", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Dilatação

In [61]:
def thick_font(image):
    image = cv2.bitwise_not(image)
    kernel = np.ones((2,2), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)


In [62]:
dilated_image = thick_font(no_noise)
cv2.imwrite("temp/dilated_image.jpg", dilated_image)

True

In [63]:
img = cv2.imread("temp/dilated_image.jpg")
cv2.imshow("gray", img)
cv2.waitKey(0)
cv2.destroyAllWindows()